————————————
Number = 1
전체 틀 : 
1. While True : 
2. 드라이버 호출 및 접속
If number == 1 : (첫번째 턴이면)
driver.get(start_date)
Number =0
날짜 = start_date


Else : 
driver.get(‘날짜’)

1. 페이지 맨 아래까지 내리기 
2. 페이지 파싱
Try : 
1. for문으로 돌면서 항목별로 
- 이미지 src수집
- 순위
- 댓글 갯수
- 제목
- 언론사
- 내용 간략 설명
수집
Except : 
Pass

5. for문 끝나면 : 
6. If 날짜 == end_date : break
7. Else : 
8. Try : 
9. A class=btn_date btn_next인 태그 찾고[‘href’]에서 맨 마지막 날짜 정보 찾고, ‘날짜’ 변수에 담는다. 
10. driver.quit()
11. Except : 
12. Break

f.close()
Driver.quit()

데이터프레임에 저장 후
이미지 파일도 엑셀에 함께 쓰기 

끝


—————————————





In [37]:
import selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
from selenium.webdriver.common.keys import Keys

import openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from PIL import Image
from io import BytesIO
import requests

In [38]:
print('-'*100)
print('다음 뉴스 댓글 많은 수 랭킹 크롤러')
print('다음 뉴스 조회 시작일 부터 조회 종료일까지의 댓글 많은 수 랭킹 수집하는 크롤러 입니다')
print('-'*100)

def start_date() : 
    while True : 
        try : 
            start_date = int(input('1.조회 시작 일자를 입력하세요(예:20201221) :'))
            if len(str(start_date)) != 8 : 
                raise ValueError
            elif str(start_date)[0:4] not in ['2019','2020','2021'] : 
                raise ValueError
            elif str(start_date)[4:6] == '02' : 
                if int(str(start_date)[6:8]) >= 28 : 
                    raise TypeError
            break
        except : 
            print('잘못 입력하셨습니다. 다시 입력하세요')
            pass
    return start_date

def end_date() : 
    while True : 
        try : 
            end_date = int(input('2.조회 종료 일자를 입력하세요(예:20201221) :'))
            if len(str(end_date)) != 8 : 
                raise ValueError
            elif str(end_date)[0:4] not in ['2019','2020','2021'] : 
                raise ValueError
            elif str(end_date)[4:6] == '02' : 
                if int(str(end_date)[6:8]) >= 28 : 
                    raise TypeError
            break
        except : 
            print('잘못 입력하셨습니다. 다시 입력하세요')
            pass
    return end_date


start_date = start_date()
end_date = end_date()

print('데이터를 수집해보겠습니다===========================')

----------------------------------------------------------------------------------------------------
다음 뉴스 댓글 많은 수 랭킹 크롤러
다음 뉴스 조회 시작일 부터 조회 종료일까지의 댓글 많은 수 랭킹 수집하는 크롤러 입니다
----------------------------------------------------------------------------------------------------
1.조회 시작 일자를 입력하세요(예:20201221) :20201101
2.조회 종료 일자를 입력하세요(예:20201221) :20210308
데이터를 수집해보겠습니다===========================


In [39]:
f_dir = '/Users/kibeomkim/Desktop/웹크롤링_결과_데이터/'
now = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(now.tm_year, now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
os.makedirs(f_dir+s+'-'+'다음뉴스 댓글순 랭킹')
os.chdir(f_dir+s+'-'+'다음뉴스 댓글순 랭킹')

In [40]:
phase = 1
img_list1 = []
rank_list1 = []
ddat_list1 = []
title_list1 = []
press_list1 = []
contents_list1 = []
no = 0
number = 0
s_time = time.time()

while True : 
    path = '/Users/kibeomkim/Desktop/Chromedriver_88/chromedriver'
    options= webdriver.ChromeOptions()
    options.add_argument('User-Agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/14.0.2 Safari/605.1.15')
    driver = webdriver.Chrome(path, options=options)
    
    if phase == 1 : 
        driver.get('https://news.daum.net/ranking/bestreply?regDate=%s' %str(start_date))
        time.sleep(5)
        phase = 0
        date = str(start_date)
    else : 
        driver.get('https://news.daum.net/ranking/bestreply?regDate=%s' %date)
        time.sleep(5)
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try : 
        news_list = soup.find('ul', 'list_news2').find_all('li')
        os.makedirs(f_dir+s+'-'+'다음뉴스 댓글순 랭킹/'+date)
        os.chdir(f_dir+s+'-'+'다음뉴스 댓글순 랭킹/'+date)
        f = open(date+'-'+'다음뉴스랭킹.txt', 'a', encoding='utf-8')
        
        for news in news_list : 
            try : 
                img_src = news.find('img')['src']
                img_list1.append(img_src)
            except : 
                img_list1.append(None)
            
            f.write('-'*100+'\n')
            print('-'*100)
            
            rank = news.find('span', 'num_rank').get_text().strip()
            rank_list1.append(rank)
            f.write('1.순위 : %s' %rank+'\n')
            print('1.순위 : %s' %rank)
            
            ddat = news.find('span', 'info_rank').find_all('span')[1].get_text().strip()
            ddat_list1.append(ddat)
            f.write('2.댓글 갯수 : %s' %ddat+'\n')
            print('2.댓글 갯수 : %s' %ddat)
            
            title = news.find('strong', 'tit_thumb').find('a').get_text().replace('\n', "").strip()
            title_list1.append(title)
            f.write('3.뉴스 제목 : %s' %title+'\n')
            print('3.뉴스 제목 : %s' %title)
            
            press_name = news.find('strong', 'tit_thumb').find('span', 'info_news').get_text().replace('\n', "").strip()
            press_list1.append(press_name)
            f.write('4.언론사 이름 : %s' %press_name+'\n')
            print('4.언론사 이름 : %s' %press_name)
            
            contents = news.find('span', 'link_txt').get_text().replace('\n', "").strip()
            contents_list1.append(contents)
            f.write('5.뉴스 내용 요약 : %s' %contents+'\n')
            print('5.뉴스 내용 요약 : %s'%contents)
            
            no += 1
            
        f.close()
        daum = pd.DataFrame({
            '순위' : rank_list1,
            '댓글 갯수' : ddat_list1,
            '뉴스 제목' : title_list1,
            '언론사 이름' : press_list1,
            '뉴스 내용 요약' : contents_list1}
        )
        daum.to_csv(date+'-'+'다음뉴스랭킹.csv',encoding='utf-8-sig',index=False)
        daum.to_excel(date+'-'+'다음뉴스랭킹.xlsx',index=False)
        number += 1
        
        rank_list1 = []
        ddat_list1 = []
        title_list1 = []
        press_list1 = []
        contents_list1 = []
        
        #엑셀에 이미지 함께 저장
        wb = openpyxl.load_workbook(date+'-'+'다음뉴스랭킹.xlsx')
        ws = wb.worksheets[0]
        for a in range(2, 52) : 
            ws.row_dimensions[a].height = 100
        
        ws.column_dimensions['C'].width = 80
        
        nor = 2
        img_no = 0
        for a in img_list1 : 
            try : 
                res = requests.get(a)
                image_data = BytesIO(res.content)
                img = openpyxl.drawing.image.Image(image_data)
                ws.add_image(img, anchor='C%s' %nor)
                nor +=1
                img_no += 1
                print('%s번째 이미지를 저장했습니다' %img_no)
            except : 
                nor += 1
                print('원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다')
                continue
        wb.save(date+'-'+'다음뉴스랭킹.xlsx')
        print('엑셀에 이미지 저장 완료')
        img_list1 = []

        

    except : 
        pass
    os.chdir(f_dir+s+'-'+'다음뉴스 댓글순 랭킹')
    if date == str(end_date) : 
        break
    else : 
        try : 
            date = soup.find('a', 'btn_date btn_next')['href'][-8:]
            driver.quit()
        except : 
            break

driver.quit()
e_time = time.time()
total_time = e_time - s_time

print('-'*100)
print('그룹지어 %s건, 개별로 %s건의 다음 뉴스기사 텍스트 데이터를 수집했습니다' %(number,no))
print('총 소요시간은 %s초 입니다' %round(total_time,2))
print('='*100)

----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 10,971
3.뉴스 제목 : "커밍아웃 검사, 사표 받아라" 靑청원 이틀만 20만 돌파(종합)
4.언론사 이름 : 뉴시스
5.뉴스 내용 요약 : [서울=뉴시스] 안채원 김재환 기자 = 추미애 법무부 장관의 수사지휘권과 인사권, 감찰권 행사 등을 비판한 검사들에게 사표를 받으라는 청와대 국민청원이 게시 이틀 ...
----------------------------------------------------------------------------------------------------
1.순위 : 2
2.댓글 갯수 : 9,053
3.뉴스 제목 : 나경원 "제 아들은 곧 입대"..이중국적 아들 둔 조국 압박
4.언론사 이름 : 뉴시스
5.뉴스 내용 요약 : [서울=뉴시스] 박준호 기자 = 나경원 전 국민의힘 의원은 1일 "혼탁한 시국에 제 아들은 곧 군 입대를 한다"며 "대한민국 국민으로서 당연히 이행해야 할 의무라지만, ...
----------------------------------------------------------------------------------------------------
1.순위 : 3
2.댓글 갯수 : 6,682
3.뉴스 제목 : 세금 무서워 고향집 내놓고..양평·속초는 '세컨드하우스' 급매 속출
4.언론사 이름 : 한국경제
5.뉴스 내용 요약 : 경북 김천시 율곡동의 한 공인중개업소에는 매수자를 찾는 광고문이 창문 가득 붙어 있다. 구청 단속도 아랑곳하지 않고 대로변에 급매물 광고판을 세운 곳도 있다. 한 ...
----------------------------------------------------------------------------------------------------
1.순위 : 4

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 15,988
3.뉴스

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 7,970
3.뉴스 

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 6,998
3.뉴스 

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
50번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 13,986
3.뉴스 제목 : 정경심에 징역 

...

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
50번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 10,765
3.뉴스 제목 : 방상훈 조선일보

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
50번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 6,892
3.뉴스 제목 : 추미애 전 장관,

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수 : 11,020
3.뉴스

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
원래 사이트에 이미지 주소가 없어 저장하지 못 했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
1.순위 : 1
2.댓글 갯수

1번째 이미지를 저장했습니다
2번째 이미지를 저장했습니다
3번째 이미지를 저장했습니다
4번째 이미지를 저장했습니다
5번째 이미지를 저장했습니다
6번째 이미지를 저장했습니다
7번째 이미지를 저장했습니다
8번째 이미지를 저장했습니다
9번째 이미지를 저장했습니다
10번째 이미지를 저장했습니다
11번째 이미지를 저장했습니다
12번째 이미지를 저장했습니다
13번째 이미지를 저장했습니다
14번째 이미지를 저장했습니다
15번째 이미지를 저장했습니다
16번째 이미지를 저장했습니다
17번째 이미지를 저장했습니다
18번째 이미지를 저장했습니다
19번째 이미지를 저장했습니다
20번째 이미지를 저장했습니다
21번째 이미지를 저장했습니다
22번째 이미지를 저장했습니다
23번째 이미지를 저장했습니다
24번째 이미지를 저장했습니다
25번째 이미지를 저장했습니다
26번째 이미지를 저장했습니다
27번째 이미지를 저장했습니다
28번째 이미지를 저장했습니다
29번째 이미지를 저장했습니다
30번째 이미지를 저장했습니다
31번째 이미지를 저장했습니다
32번째 이미지를 저장했습니다
33번째 이미지를 저장했습니다
34번째 이미지를 저장했습니다
35번째 이미지를 저장했습니다
36번째 이미지를 저장했습니다
37번째 이미지를 저장했습니다
38번째 이미지를 저장했습니다
39번째 이미지를 저장했습니다
40번째 이미지를 저장했습니다
41번째 이미지를 저장했습니다
42번째 이미지를 저장했습니다
43번째 이미지를 저장했습니다
44번째 이미지를 저장했습니다
45번째 이미지를 저장했습니다
46번째 이미지를 저장했습니다
47번째 이미지를 저장했습니다
48번째 이미지를 저장했습니다
49번째 이미지를 저장했습니다
50번째 이미지를 저장했습니다
엑셀에 이미지 저장 완료
----------------------------------------------------------------------------------------------------
그룹지어 128건, 개별로 6400건의 다음 뉴스기사 텍스트 데이터를 수집했습니